In [6]:
import pandas as pd
import matplotlib.pyplot as plt
from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.tsa.stattools import adfuller
from statsmodels.tsa.stattools import acf, pacf
import numpy as np

# Exploración de Datos

In [21]:
df = pd.read_excel('consumo.xlsx', engine='openpyxl')
df = df[['Fecha', 'Gasolina superior', 'Gasolina regular', 'Diesel']]

In [25]:
df = df[:269]
df

,Fecha,Gasolina superior,Gasolina regular,Diesel
0,2000-01-01 00:00:00,308157,202645,634667
1,2000-02-01 00:00:00,307766,205531,642381
2,2000-03-01 00:00:00,331910,229500,699807
3,2000-04-01 00:00:00,315648,210680,586804
4,2000-05-01 00:00:00,319668,208164,656948
...,...,...,...,...
264,2022-01-01 00:00:00,622475,684004,1.29426e+06
265,2022-02-01 00:00:00,592075,660944,1.27576e+06
266,2022-03-01 00:00:00,599582,703128,1.3452e+06
267,2022-04-01 00:00:00,566128,807440,1.29416e+06
